## Introduction



We&rsquo;re thinking about the problem of finding the cheapest possible
nutritious diet.  Last time we argued that this problem could be
expressed as a *linear program*
$$
    \min_x p'x
$$

such that
$$\begin{bmatrix}
      A\\
      -A
   \end{bmatrix}x \geq \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix},$$

where $p$ is a vector of prices, $A$ is a matrix that maps
vectors of quantities of food into vectors of nutrients, and where
$b_{min}$ and $b_{max}$ are respectively dietary minimums
and maximums of different nutrients.  As above, we will sometimes stack these
objects, obtaining
$$
      \tilde{A} = \begin{bmatrix}
                        A_{min}\\
                        -A_{max}
                      \end{bmatrix}
  $$
and
$$
      \tilde{b} = \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix}
  $$

Our job in this notebook: Specify the objects required by the linear
program $(p,\tilde{A},\tilde{b})$, then have the computer solve the problem for us.



## USDA Food Central DataBase



The USDA maintains a database of nutritional information, where
different kinds of food are identified by an FDC number.  They do
not provide any data on prices.  

To look up nutritional information, use api provided by the USDA at
[https://fdc.nal.usda.gov/](https://fdc.nal.usda.gov/).   You should sign up for a
free api key (see directions on page), then add that key here in
place of &ldquo;DEMO<sub>KEY</sub>&rdquo;.



In [1]:
apikey = "Y7y8evhwYYL589qVEZzdHBCyKGeBkNdDWQfnWyIV"  # Replace with a real key!  "DEMO_KEY" will be slow...

### Looking up foods



I&rsquo;ve written a little module `fooddatacentral`.  Install it (only once!), along with other requirements.



In [2]:
%pip install -r requirements.txt --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.5/38.5 MB 25.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 979.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 kB 8.6 MB/s eta 0:00:0000:01
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 9.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: Pint
    Found existing installation: Pint 0.17
    Uninstalling Pint-0.17:
      Successfully uninstalled Pint-0.17
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstallin

This module offers some simple methods

-   `search`
-   `nutrients`
-   `units`



### FDC Search



Here&rsquo;s a little code to help look up FDC codes for foods of
different descriptions.



In [4]:
import fooddatacentral as fdc

fdc.search(apikey,"insect")

,fdcId,description,dataType,gtinUpc,publishedDate,brandOwner,brandName,ingredients,marketCountry,foodCategory,...,tradeChannels,allHighlightFields,score,microbes,foodNutrients,finalFoodInputFoods,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds
0,2642964,INSECT LOLLIPOP,Branded,810051802849,2023-09-14,Sundia Corporation,AMOS,"SUGAR, GLUCOSE SYRUP, MALTODEXTRIN, WATER, ART...",United States,Candy,...,[NO_TRADE_CHANNEL],,386.50522,[],"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[]
1,2642962,INSECT LOLLIPOP,Branded,810051802702,2023-09-14,Sundia Corporation,AMOS,"SUGAR, GLUCOSE SYRUP, MALTODEXTRIN, WATER, ART...",United States,Candy,...,[NO_TRADE_CHANNEL],,386.50522,[],"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[]
2,2642963,INSECT LOLLIPOP,Branded,810051802757,2023-09-14,Sundia Corporation,AMOS,"SUGAR, GLUCOSE SYRUP, MALTODEXTRIN, WATER, ART...",United States,Candy,...,[NO_TRADE_CHANNEL],,386.50522,[],"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[]
3,2048018,"HILCO, INSECT POPPER WITH CANDY",Branded,083933218414,2021-10-28,Hilco LLC,HILCO,"DEXTROSE, GLUCOSE (CORN SYRUP), CITRIC ACID, C...",United States,Candy,...,[NO_TRADE_CHANNEL],,257.43810,[],"[{'nutrientId': 1003, 'nutrientName': 'Protein...",[],[],[],[],[]


### FDC Nutrients



Once we know the `fdc_id` of a particular food we can look up a
variety of information on it.  We start with nutrients



In [5]:
id = 2642963    # Put an FDC ID HERE!
fdc.nutrients(apikey,fdc_id=id)

,Quantity,Units
"Calcium, Ca",0.00,mg
"Potassium, K",0.00,mg
Vitamin D (D2 + D3),0.00,µg
"Sodium, Na",0.00,mg
"Iron, Fe",0.00,mg
"Fatty acids, total trans",0.00,g
Total lipid (fat),0.00,g
Energy,400.00,kcal
"Sugars, added",73.30,g
"Carbohydrate, by difference",100.00,g


## Prices



Now, let&rsquo;s begin thinking about constructing the objects we need for
the linear program.  Start with specifying $p$, the vector of prices.  

Also note that some kinds of foods need to have unit weights (in
grams) supplied under &ldquo;Units&rdquo;; e.g., extra large eggs are taken to
each weigh 56g.  These conversions can also often be found on the USDA
FDC website.  

Food is purchased in particular units (gallons, pounds, grams).  And
in some cases the natural units are things like donuts or eggs, in
which case we may need to define our  own units (see the example of
&ldquo;xl<sub>egg</sub>&rdquo; below).  New units can be added to a file `.unitsrc` in your home directory.



### Example: Stigler&rsquo;s Foods



In his 1945 paper George Stigler constructed a subsistence diet
chosen from 14 different goods (see Table B in [Stigler 1945](https://www.jstor.org/stable/pdf/1231810.pdf)), with
prices reported for the years 1939 & 1944.

I&rsquo;ve looked up more recent prices for these same goods, and recorded
these at
[https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/](https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/), in a sheet called &ldquo;Stigler Table B (2022 Prices)&rdquo;

The code below allows us to collect data on different kinds of food
with their prices from google spreadsheets.

In this case, we use a function from a module I&rsquo;ve written,
 `eep153_tools.sheets`, to read the price data for the
Stigler goods.



In [7]:
%pip install python-gnupg

  Using cached python_gnupg-0.5.2-py2.py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from eep153_tools.sheets import read_sheets

df = read_sheets("1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A",sheet='Stigler Table B (2022 Prices)')

df = df.set_index('Food')

df

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:165: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


,Quantity,Units,Price,Date,Location,FDC
Food,,,,,,
Wheat Flour,80.0,oz,3.79,[2022-2-28],Safeway,2432946
Wheat Cereal,16.4,oz,3.49,[2022-2-28],Safeway,2343964
Corn Meal,80.0,oz,4.49,[2022-2-28],Safeway,2079814
Rolled Oats,42.0,oz,3.99,[2022-2-28],Safeway,2478125
Evaporated Milk,12.0,oz,1.99,[2022-2-28],Safeway,1889123
Cabbage,1.0,lbs,1.29,[2022-2-28],Safeway,169975
Potatoes,1.0,lbs,1.49,[2022-2-28],Safeway,576920
Spinach,1.0,oz,0.25,[2022-2-28],Safeway,168462
Sweet Potatoes,1.0,lbs,1.99,[2022-2-28],Safeway,2427040


### Example: My Shopping Trip



Here&rsquo;s an example of describing some different kinds of food, along with
data on food prices.  This is all just based on a trip I took to the
grocery store, except that I&rsquo;ve used the USDA database to look up FDC
numbers.  Note that we may need extra information to map some units
into weights.  For example, I still need to weigh a crumpet.



#### Trip to Monterey Market



In [10]:
import pandas as pd
from eep153_tools.sheets import read_sheets

df = read_sheets('https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/',sheet="Ligon's Shopping Trip")

df = df.set_index('Food')

df

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:165: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


,Quantity,Units,Price,Date,Location,FDC
Food,,,,,,
"Milk, 2% fat",1.00,gallon,4.99,[2019-09-14 Sat],"Monterey Market, Berkeley",2485214
"Eggs, extra large",12.00,xl_egg,3.59,[2019-09-14 Sat],"Monterey Market, Berkeley",747997
Crumpets,6.00,crumpet,3.19,[2019-09-14 Sat],"Monterey Market, Berkeley",1930285
Bananas,1.00,pound,3.15,[2019-09-14 Sat],"Monterey Market, Berkeley",173944
"Carrots, Organic",2.00,pound,2.29,[2019-09-14 Sat],"Monterey Market, Berkeley",170393
Cauliflower,2.51,pound,4.24,[2019-09-14 Sat],"Monterey Market, Berkeley",169986
"Endive, Red",1.26,pound,6.27,[2019-09-14 Sat],"Monterey Market, Berkeley",168412
"Figs, black mission",1.00,pound,4.98,[2019-09-14 Sat],"Monterey Market, Berkeley",438223
"Leeks, Organic",1.00,pound,1.29,[2019-09-14 Sat],"Monterey Market, Berkeley",169246


### Units & Prices



Now, the prices we observe can be for lots of different quantities and
 units.  The FDC database basically wants everything in either hundreds
 of grams (hectograms) or hundreds of milliliters (deciliters).  

Sometimes this conversion is simple; if the price we observe is for
something that weighs two kilograms, that&rsquo;s just 20 hectograms.
Different systems of weights and volumes are also easy; a five pound
bag of flour is approximately 22.68 hectograms.  

Othertimes things are more complicated.  If you observe the price of a
dozen donuts, that needs to be converted to hectograms, for example.  

A function `units` in the [fdc](fooddatacentral.py) module accomplishes this conversion
for many different units, using the `python` [pint module](https://pint.readthedocs.io/en/latest/).  A file
[~/.units.rc](Data/food_units.txt) can be edited to deal with odd cases such as
donuts, using a format described in the `pint` [documentation](https://pint.readthedocs.io/en/latest/advanced/defining.html).

Here&rsquo;s an example of the usage of `fooddatacentral.units`:



In [11]:
# Try your own quantities and units.
# If units are missing try adding to ~/.unitsrc

print(fdc.units(5,'lbs'))
print(fdc.units(1,'gallon'))
print(fdc.units(2,'tea_bag'))
print(fdc.units(12,'donut'))

22.679618500000004 hectogram
37.85411783999999 deciliter
0.06 hectogram
4.4225256075 hectogram


Now, use the `units` function to convert all foods to either
 deciliters or hectograms, to match FDC database:



In [12]:
import fooddatacentral as fdc

# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['FDC Price'].min()

Prices

/srv/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


Food
Bananas                    0.6944561258823643 / hectogram
Beef Liver                 2.1076192264874294 / hectogram
Carrots, Organic           0.2524292902016848 / hectogram
Cauliflower               0.37241433930831913 / hectogram
Crumpets                                 nan / milliliter
Eggs, extra large          0.5342261904761904 / hectogram
Endive, Red                1.0970622094437954 / hectogram
Figs, black mission        1.0979020656806904 / hectogram
Leeks, Organic            0.28439631821849204 / hectogram
Lettuce, Little Gem         1.318364327865568 / hectogram
Marmite                      9.61215463126066 / hectogram
Milk, 2% fat               0.1318218541267161 / deciliter
Mushrooms, King Oyster     2.6455471462185307 / hectogram
Onion, yellow             0.08598028225210225 / hectogram
Orange juice              0.47445300603523477 / deciliter
Parsnip                   0.43651527912605753 / hectogram
Potato, marble mix         0.5709972590588328 / hectogram
Potato, r

## Mapping to Nutrients



Next we want to build the matrix $A$, which maps quantities of food
 into nutrients.  We have a list of foods with prices.  Do lookups on USDA database
 to get nutritional information.



In [13]:
import warnings

D = {}
count = 0
for food in  df.index:
    try:
        FDC = df.loc[df.index==food,:].FDC.values[0]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
        print(food)
    except AttributeError:
        warnings.warn(f"Couldn't find FDC Code {FDC} for food {food}.")

D = pd.DataFrame(D,dtype=float)

D

Milk, 2% fat
Eggs, extra large
Crumpets
Bananas
Carrots, Organic
Cauliflower
Endive, Red
Figs, black mission
Leeks, Organic
Lettuce, Little Gem
Mushrooms, King Oyster
Onion, yellow
Orange juice
Parsnip
Potato, marble mix
Rhubarb
Potato, russet
Squash, Zucchini
Marmite
Beef Liver


,"Milk, 2% fat","Eggs, extra large",Crumpets,Bananas,"Carrots, Organic",Cauliflower,"Endive, Red","Figs, black mission","Leeks, Organic","Lettuce, Little Gem","Mushrooms, King Oyster","Onion, yellow",Orange juice,Parsnip,"Potato, marble mix",Rhubarb,"Potato, russet","Squash, Zucchini",Marmite,Beef Liver
"Ergosta-5,7-dienol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Ergosta-7,22-dienol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alanine,NaN,0.714,NaN,0.04,0.113,0.116,0.062,NaN,0.074,NaN,NaN,0.021,NaN,NaN,NaN,NaN,0.08,0.063,NaN,NaN
"Alcohol, ethyl",NaN,NaN,NaN,0.00,0.000,0.000,0.000,NaN,0.000,NaN,NaN,0.000,NaN,NaN,NaN,0.00,0.00,0.000,0.00,0.00
Amino acids,NaN,0.000,NaN,0.00,0.000,0.000,0.000,NaN,0.000,NaN,0.0000,0.000,NaN,0.00,NaN,0.00,0.00,0.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Menaquinone-4),NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),NaN,NaN,NaN,0.50,13.200,15.500,231.000,NaN,47.000,NaN,NaN,0.400,NaN,22.50,NaN,29.30,2.00,4.300,0.00,3.90
Vitamins and Other Components,NaN,0.000,NaN,0.00,0.000,0.000,0.000,NaN,0.000,NaN,0.0000,0.000,NaN,0.00,0.00,0.00,0.00,0.000,NaN,NaN
Water,NaN,86.300,NaN,74.91,88.290,92.070,93.790,NaN,83.000,NaN,88.1300,89.110,NaN,79.53,83.29,93.61,74.45,94.790,40.90,61.50


## Dietary Requirements



We&rsquo;ve figured out some foods we can buy, the nutritional content of
those foods, and  the price of the foods.  Now we need to say
something about nutritional requirements, and construct the vectors
$b_{min}$ and $b_{max}$.   Our data for this is based
on  US government recommendations available at
[https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf](https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf)

I&rsquo;ve put some of these data into a google spreadsheet at
[https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/](https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/). 
Note that we&rsquo;ve tweaked the nutrient labels to match those in the FDC
data.

We&rsquo;ve broken down the requirements into three different tables.  The
first is *minimum* quantities that we need to  satisfy.  For example,
this table tells us that a 20 year-old female needs at least 46 grams
of protein per day.



In [14]:
RDIs = read_sheets('https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/')

bmin = RDIs['diet_minimums'].set_index('Nutrition')

# Drop string describing source
bmin = bmin.drop('Source',axis=1)

bmin

Key available for students@eep153.iam.gserviceaccount.com.


/srv/conda/lib/python3.9/site-packages/eep153_tools/sheets.py:150: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  D[w.title]=df.apply(lambda x: pd.to_numeric(x,errors='ignore'))


,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


This next table specifies *maximum* quantities.  Our 20 year-old
female shouldn&rsquo;t have more than 2300 milligrams of sodium per day.



In [1]:
bmax = RDIs['diet_maximums'].set_index('Nutrition')

# Drop string describing source
bmax = bmax.drop('Source',axis=1)

bmax

## Putting it together



Here we take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.
Recall that the mathematical problem we&rsquo;re trying to solve is
$$
    \min_x p'x
$$
such that
$$
     \tilde{A}x \geq \tilde{b}
$$



### Objective function ($p$)



If we buy a bag of groceries with quantities given by $x$, the total
cost of the bag of groceries is the inner product of prices and
quantities.  Since we&rsquo;ve converted our units above, this gives us a
vector of prices where quantities are all in 100 g or ml units.



In [1]:
p = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = p.index.intersection(D.columns)
p = p[use]

p

### Nutrient Mapping Matrix ($A$)



The matrix $A$ maps a bag of groceries $x$ into nutrients, but we
don&rsquo;t need to keep track of nutrients for which we don&rsquo;t have
contraints.



In [1]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Maximum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])

A

### Constraint vector ($b$)



Finally, the right hand side vector $b$ in the expression
$$
    Ax\geq b
$$



In [1]:
b = pd.concat([bmin,-bmax]) # Note sign change for max constraints

b

## Solving the problem



First, we find a solution to the problem



In [1]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

## Choose sex/age group!
group = "F 19-30"

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(p, -A, -b[group], method='highs')

result

Let&rsquo;s interpret this.  Start with the cost of the solution:



In [1]:
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")

Next, what is it we&rsquo;re actually eating?



In [1]:
# Put back into nice series
diet = pd.Series(result.x,index=p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

Given this diet, what are nutritional outcomes?



In [1]:
tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b[group])})
print("\nWith the following nutritional outcomes of interest:")
tab

Finally, what are the constraints that bind?  Finding a less expensive
diet might involve finding less expensive sources for these particular nutrients.



In [1]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())